In [ ]:
import pandas as pd


# Reade the data from excel copy

In [ ]:
df = pd.read_clipboard()

In [ ]:
df.shape

In [ ]:
start_date = "2020-11-01"
df = df.rename(columns={"SignalTime": "Date"}).set_index("Date")
df.index = pd.to_datetime(df.index)
df = df.loc[start_date:]

# Verify Wide VA

In [ ]:
mask = ((df["FtValueAreaHigh"] - df["FtValueAreaLow"]) / df["FtAtr"] < 0.5)
df.loc[mask].empty

# Price "above" Wide VA up and "below" Wide VA down 

In [ ]:
up_mask = (
    (df["Direction"] == "UP") &
    (df["FtSignalLow"] > (df["FtValueAreaHigh"] - df["BgAtr"] * 0.4))
)
down_mask = (
    (df["Direction"] == "DOWN") &
    (df["FtSignalHigh"] < (df["FtValueAreaLow"] + df["BgAtr"] * 0.4))
)
# df.loc[up_mask | down_mask].to_clipboard()
df_outside_va = df.loc[up_mask | down_mask].copy(deep=True)

### Identify some attributes for the behavior
We will use the bg close move or the bg MAE MFE relationship

In [ ]:
# use VA as an identifier
up_mask = (
    (df_outside_va["Direction"] == "UP") &
    (df_outside_va["BgP1MFE"] < df_outside_va["BgP2MFE"]) &
    (df_outside_va["BgP2MFE"] < df_outside_va["BgP3MFE"]) &
    (df_outside_va["BgP2MAE"] >= df_outside_va["BgP3MAE"]) 
)
down_mask = (
    (df_outside_va["Direction"] == "DOWN") &
    (df_outside_va["BgP1MFE"] > df_outside_va["BgP2MFE"]) &
    (df_outside_va["BgP2MFE"] > df_outside_va["BgP3MFE"]) &
    (df_outside_va["BgP2MAE"] <= df_outside_va["BgP3MAE"]) 
)

df_outside_va["BgMoveMFEMAE"] = 0
df_outside_va.loc[(up_mask | down_mask), "BgMoveMFEMAE"] = 1

In [ ]:
# use VA as an identifier
up_mask = (
    (df_outside_va["Direction"] == "UP") &
    (df_outside_va["BgP1Close"] < df_outside_va["BgP2Close"]) &
    (df_outside_va["BgP2Close"] < df_outside_va["BgP3Close"]) 
)
down_mask = (
    (df_outside_va["Direction"] == "DOWN") &
    (df_outside_va["BgP1Close"] > df_outside_va["BgP2Close"]) &
    (df_outside_va["BgP2Close"] > df_outside_va["BgP3Close"])  
)

df_outside_va["BgMoveClose"] = 0
df_outside_va.loc[(up_mask | down_mask), "BgMoveClose"] = 1

In [ ]:
mask = (
    (df_outside_va["BgMoveClose"]  == 1) |
    (df_outside_va["BgMoveMFEMAE"] == 1)
)
df_outside_va["BgMove"] = 0 
df_outside_va.loc[mask, "BgMove"] = 1 

In [ ]:
df_outside_va.drop_duplicates(subset=["Ticker", "BgBarNumber"]).to_clipboard()